## Model compression

This file will **compress the model** generated by the transfer learning to a TF Lite model.

To run this code you need to have a model ready and the test dataset folder set, you can download it here: https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip

you then need to put it in the `Dataset` folder to have this hierarchy:

* Dataset
    - Test
        - paper
        - rock
        - scissors

In [1]:
import numpy as np
import tensorflow as tf
import os
import time
import PIL.Image as Image

If you need, you can change the path to your `TFModel`, `TFLiteModel` and `Dataset` folder here.

In [2]:
path_to_model_folder = "saved_models/TFModel/"
path_to_tf_model_folder = "saved_models/TFLiteModel/"
dataset_test = "Dataset/Test"

This cell will load your saved model and convert it to a TF lite model.

It then saves the TF lite model to the same folder as the non compressed model.

In [3]:
converter = tf.lite.TFLiteConverter.from_saved_model(path_to_model_folder + "RPS_Fine_Tuned_Model") # path to the SavedModel directory
tflite_model = converter.convert()


# Save the model
with open(path_to_tf_model_folder + "RPS_Fine_Tuned_Model.tflite", 'wb') as f:
  f.write(tflite_model)

## Testing the TF lite model

The following code will test the compressed model to see its accuracy.

TF lite models only accept **one image at the time**, we therefore have to load all our images manually.

Also, our dataset is compose of png images that have alpha channel, but our **model do not accept images that have an alpha channel**.

This function **load all the images** in the dataset test folder, **remove the alpha channel** and **put it in an array**.

In [4]:
IMAGE_SHAPE = (224, 224)

def load_dataset():
    X_test = []
    y_test = []

    for classes in os.listdir(dataset_test):
        if classes == ".gitkeep":
            continue
        classes_path = os.path.join(dataset_test, classes)
        for image_name in os.listdir(classes_path):
            img = Image.open(os.path.join(classes_path, image_name)).resize(IMAGE_SHAPE)
            alpha = img.convert('RGBA').split()[-1]
            bg = Image.new("RGB", img.size, (255, 255, 255))
            bg.paste(img, mask=alpha)
            X_test.append(np.array(bg, dtype=np.float32)/255.0)
            y_test.append(classes)
    return X_test, y_test

This cell creates an interpreter with the TF lite model and it gives us the **inputs and outputs** of the model.

In [5]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']

X_test, y_test = load_dataset()

The model will now predict the classes for all images in our test set.

This cell also keeps track of the time needed for each classification.

In [6]:
predicted_result = []
classifing_speed = []

for i in range(len(X_test)):
    start_time = time.time()
    test = np.array(X_test[i][np.newaxis, ...])
    interpreter.set_tensor(input_details[0]['index'], test)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    classifing_speed.append(time.time() - start_time)
    predicted_result.append(np.argmax(output_data[0], axis = -1))

In [7]:
average_time_per_image = sum(classifing_speed) / len(predicted_result)
print("Average time to classify an image is: ", average_time_per_image)

Average time to classify an image is:  0.033375687496636504


In [8]:
accuracy = 0.0

for i in range(len(predicted_result)):
    if ({0:'paper', 1:'rock', 2:'scissors'}[predicted_result[i]] == y_test[i]):
        accuracy += 1

accuracy = accuracy / len(predicted_result)
print("Accuracy of TFlite model is: ", accuracy)

Accuracy of TFlite model is:  0.8225806451612904


## Result

The TF lite mode takes 0.03 second to classify an image which is really good.

It has an accuracy of 0.82, it lost 0.1 of accuracy with the compression but it is usable.